## Generating Flag Matroids

In this notebook, we determine all rank-$(1,2,3)$ flag matroids on $[4]$ up to $\mathfrak{S}_2 \times \mathfrak{S}_4$-symmetry. For each flag matroid $\vec{\mathsf{Q}}$ whose polytope is full-dimensional, we also determine the matroids of its internal facets, i.e., facets that are not contained in the boundary of the 3-dimensional permutahedron. 

In [1]:
using Oscar
pm = Polymake

 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.10.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


Polymake

### Generating all matroids on $[n]$ from a list of $\mathfrak{S}_n$-orbit representatives

**Function**: ```matroidsFromFile```

*Input*: ```fname = String, n = Int64```

*Output*: ```Dict{Int64,Vector{Any}}```

*Description*: Reads a file containing the matroids on $[n]$ where each line records a matroid as a string of ```*``` (bases) and ```0``` (nonbases) ordered by revlex. This function outputs a dictionary where the keys are the possible ranks, and the value at a rank ```d``` is the list of ($\mathfrak{S}_n$-orbit representatives of) $(d,n)$-matroids. 

---

**Function**: ```matroidsBasesFromFile```

*Input*: ```fname = String, n = Int64```

*Output*: ```Dict{Int64,Vector{Any}}```

*Description*: Reads a file containing the matroids on $[n]$ where each line records a matroid as a string of ```*``` (bases) and ```0``` (nonbases) ordered by revlex. This function outputs a dictionary where the keys are the possible ranks, and the value at a rank ```d``` is the list of the bases of ($\mathfrak{S}_n$-orbit representatives of) $(d,n)$-matroids. 

---

**Function**: ```permOnBases```

*Input*: ```Bs = Vector{Set{Int64}}, sigma = Permutation```

*Output*: ```Vector{Set{Int64}}```

*Description*: ```Bs``` is a list of bases, and this function permutes the elements within each basis by ```sigma```.

---

**Function**: ```set2Array```

*Input*: ```S = Set```

*Output*: ```Vector```

*Description*: Converts a set to a vector (elements in an arbitrary order)

---

**Function**: ```setBases2Matroid```

*Input*: ```Bs = Set{Set{Int64}}, n = Int64```

*Output*: ```Matroid```

*Description*: Given a set of bases ```Bs``` and $n$, returns the matroid on $[n]$ with these bases. 

---

**Function**: ```allMatroids```

*Input*: ```reps = Dict{Int64,Vector{Any}}, n = Int64```

*Output*: ```Dict{Int64,Any}```

*Description*: This fuunction takes as input $\mathfrak{S}_n$ orbit representatives of the matroids on $[n]$ as a dictionary like the output of ```matroidsBasesFromFile``` and returns all of the matroids on $[n]$ as a dictionary (keys = ranks, values=matroids of that rank).

---





In [2]:
function matroidsFromFile(fname, n)
    inf = open(fname);
    ls = readlines(inf);
    
    n2 = Int64(floor(n//2))
    
    Mats = Dict{Int64,Vector{Any}}(d => [] for d ∈ 1:n-1)
    
    for d ∈ 1:n2
        nCd = binomial(n,d)
        for st ∈ ls
            if length(st) == nCd
                append!(Mats[d], [pm.matroid.bases_from_revlex_encoding(st, d, n)])
                append!(Mats[n-d], [pm.matroid.bases_from_revlex_encoding(st, d, n, dual=true)])
            end
        end
    end
    
    return Mats
end

function matroidsBasesFromFile(fname, n)
    inf = open(fname);
    ls = readlines(inf);
    
    n2 = Int64(floor(n//2))
    
    Mats = Dict{Int64,Vector{Any}}(d => [] for d ∈ 1:n-1)
    
    for d ∈ 1:n2
        nCd = binomial(n,d)
        for st ∈ ls
            if length(st) == nCd                
                Bd = Vector{Set{Int64}}(pm.matroid.bases_from_revlex_encoding(st, d, n))
                Bnd = Vector{Set{Int64}}(pm.matroid.bases_from_revlex_encoding(st, d, n, dual=true))

                append!(Mats[d], [pm.to_one_based_indexing(Bd)]  )
                append!(Mats[n-d], [pm.to_one_based_indexing(Bnd)] )
            end
        end
    end
    
    return Mats
end


function permOnBases(Bs, sigma)
    return [on_sets(B, sigma) for B ∈ Bs] 
end

function matroidOrbit(Bs, n)
    orb = Set([])
    Sn = symmetric_group(n)
    SnElem = [sigma for sigma ∈ Sn]
    for sigma ∈ SnElem
        push!(orb, Set(permOnBases(Bs, sigma)))
    end
    return orb
end

function set2Array(S)
    return [i for i ∈ S]
end

function setBases2Matroid(Bs,n)
    BsVec = set2Array(Bs)
    return pm.matroid.Matroid(BASES = pm.to_zero_based_indexing(BsVec), N_ELEMENTS = n)
end


function allMatroids(reps, n)
    Ms = Dict{Int64,Any}(d => Set([]) for d ∈ 1:n-1)
    for d ∈ 1:n-1
        for M ∈ reps[d] 
            union!(Ms[d], matroidOrbit(M,n)) 
        end
    end
    return Ms
end

allMatroids (generic function with 1 method)

### Generating flag matroids

**Function**: ```flatsMatroid```

*Input*: ```M=Matroid```

*Output*: ```Vector{Set{Int64}}```

*Description*: Given a matroid $M$ returns a list of its flats. 

---

**Function**: ```isQuotient```

*Input*: ```M,N = Matroid```

*Output*: ```true``` or ```false```

*Description*: returns ```true``` if ```M``` is a quotient of ```N``` and ```false``` otherwise. 

---

**Function**: ```flagMatroids```

*Input*: ```matroidReps = Dict{Int64,Vector{Any}}, n = Int64```

*Output*: ```Vector{Vector{Set{Set{Int64}}}}```

*Description*: Returns a list of the complete flag matroids on $[n]$. Each flag matroid is a list of matroids, each matroid is recorded as a list of its bases. 

---

**Function**: ```indicatorVector```

*Input*: ```S=Set```

*Output*: ```Vector{Int64}```

*Description*: Returns the indicator vector of the set ```S```.

---

**Function**: ```unionSets```

*Input*: ```Vector{Set{Int64}}```

*Output*: ```Set{Int64}```

*Description*: returns the union of a list of sets.

---

**Function**: ```flagBases2Flags```

*Input*: ```Bs = Vector{Set{Set{Int64}}}```

*Output*: ```Vector{Vector{Set{Int64}}}```

*Description*: Given a flag matroid (as a list of list of bases) returns all flags of bases. 

---

**Function**: ```flag2Permutation```

*Input*: ```Vector{Set{Int64}}```

*Output*: ```Vector{Int64}```

*Description*: Given a complete flag of subsets, returns its corresponding permutation (as a vector $(\sigma(1),\ldots, \sigma(n))$).

---

**Function**: ```flagBases2Permutations```

*Input*: ```Bs = Vector{Set{Set{Int64}}}, n = Int64```

*Output*: ```Set{Vector{Int64}}```

*Description*: Given a flag matroid (as a list of list of bases) returns the permutations of all flags from ```Bs```. The convex hull of these points is the flag matroid polytope. 

---

**Function**: ```dualFlagPermutations```

*Input*: ```Ps = Set{Vector{Int64}}, n = Int64```

*Output*: ```Set{Vector{Int64}}```

*Description*: Given a collection of permutations, returns the collection of their duals. 

---

**Function**: ```orbitFlagMatroidPolytope```

*Input*: ```Ps = Set{Vector{Int64}}, n = Int64```

*Output*: ```Set{Set{Vector{Int64}}}```

*Description*: Given a flag matroid as a set of the vertices in its polytope, returns all such flag matroids in its $(\mathfrak{S}_2 \times \mathfrak{S}_n)$-orbit.

---

**Function**: ```flagMatroidReps```

*Input*: ```Vector{Set{Vector{Int64}}}```

*Output*: ```Dict{Set{Vector{Int64}}, Set{Set{Vector{Int64}}}}```

*Description*: Given a collection of flag matroids, returns a dictionary whose keys are particular $\mathfrak{S}_2\times \mathfrak{S}_n$-orbit representatives of flag matroids, and the values are the corresponding $\mathfrak{S}_2\times \mathfrak{S}_n$-orbits. 

---

**Function**: ```vectors2Matrix```

*Input*: ```S = Set{Vector{Int64}}```

*Output*: ```Matrix{Int64}```

*Description*: Given a set of vectors of the same length, returns a matrix whose rows are these vectors (arbitrary order). 

---

**Function**: ```setPermutations2Polytope```

*Input*: ```S = Set{Vector{Int64}}, n = Int64```

*Output*: ```Polyhedron```

*Description*: Given a set of vectors of the same length, returns their convex hull as a polyhedron. 


In [3]:
function flatsMatroid(M)
    LatticeFlatsM = M.LATTICE_OF_FLATS
    return Vector{Set{Int64}}(pm.@pm common.convert_to{Array{Set{Int}}}(LatticeFlatsM.FACES))
end

function isQuotient(M, N)
    FlatsM = Set{Set{Int64}}(flatsMatroid(M))
    FlatsN = Set{Set{Int64}}(flatsMatroid(N))
    return issubset(FlatsM, FlatsN)
end


function flagMatroids(matroidReps, n)
    Flags = Vector([[a] for a ∈ matroidReps[n-1]]);
    newFlags = Vector{Vector{Set{Set{Int64}}}}([])
    for i ∈ 2:n-1
        d = n-i
        for Bd ∈ matroidReps[d]
            Md = pm.matroid.Matroid(BASES=pm.to_zero_based_indexing([a for a ∈ Bd]), N_ELEMENTS=n)
            for Fd1 ∈ Flags
                Bd1 = last(Fd1)
                Md1 = pm.matroid.Matroid(BASES=pm.to_zero_based_indexing([a for a ∈ Bd1]), N_ELEMENTS=n)
                if isQuotient(Md,Md1)
                    push!(newFlags, push!(Fd1[:], Bd))
                end
            end
        end
        Flags = newFlags
        newFlags =  Vector{Vector{Set{Set{Int64}}}}([])
    end
    return Flags
end

function indicatorVector(S,n)
    v = Vector{Int64}([])
    for i ∈ 1:n
        if i ∈ S
            push!(v,Int64(1))
        end
        if i ∉ S
            push!(v,Int64(0))
        end
    end
    return v
end

function unionSets(Ss)
    n = length(Ss)
    U = Set{Set{Int64}}([])
    for i ∈ 1:n
        union!(U,Ss[i])
    end
    return U
end

function flagBases2Flags(Bs)
    n = length(Bs)
    Flags = [[B] for B ∈ Bs[1]]
    newFlags = Vector{Vector{Set{Int64}}}([])
    for i ∈ 2:n
        for B ∈ Bs[i]
            for Fl ∈ Flags
                if issubset(B,last(Fl)) 
                    push!(newFlags, push!(Fl[:], B))
                end
            end
        end
        Flags = newFlags
        newFlags = Vector{Vector{Set{Int64}}}([])
    end
    return Flags
end

function flag2Permutation(Fl,n)
    return sum(map((x) -> indicatorVector(x,n), Fl)) + [Int64(1) for i ∈ 1:n ]
end

function flagBases2Permutations(Bs, n)
    Flags = flagBases2Flags(Bs)
    return Set{Vector{Int64}}(map((x) -> flag2Permutation(x,n), Flags))
end

function dualFlagPermutations(Ps, n)
    return Set([ (n+1)*[1 for i ∈ 1:n] - v for v ∈ Ps     ])
end

function orbitFlagMatroidPolytope(Ps, n)
    Sn = symmetric_group(n); SnElm = [s for s ∈ Sn];
    PsArr = set2Array(Ps)
    Orb = Set{Set{Vector{Int64}}}([Set{Vector{Int64}}(Set(map((x) -> x[Vector(sigma)], PsArr))) for sigma ∈ SnElm])
    Orb2 = Set{Set{Vector{Int64}}}( [dualFlagPermutations(p,n) for p ∈ Orb]  )
    return union!(Orb,Orb2)
end


function flagMatroidReps(allPs, n)
    reps = Dict{Set{Vector{Int64}}, Set{Set{Vector{Int64}}}}([])
    for Ps ∈ allPs
        Orb = orbitFlagMatroidPolytope(Ps, n)
        if Orb ∉ values(reps)
            reps[Ps] = Orb
        end
    end
    return reps
end

function vectors2Matrix(S)
    vs = [i for i ∈ v, v ∈ S]
end
    

function setPermutations2Polytope(S,n)
    vs = [v[i] for v ∈ S, i ∈ 1:n]; 
    return convex_hull(vs)
end

function sortedFlagMatroidBases(S)
    return sort!([i for i in S], by = x->(length(x),x))
end

sortedFlagMatroidBases (generic function with 1 method)

In [4]:
function sortOrbit(Ps)
    return sort(map((x) -> sort(set2Array(x), rev=true), set2Array(Ps)), rev=true)    
end

function sortedReps(PsDict, n)
    orbits = set2Array(values(PsDict)); 
    newOrbits = [ [ [v[i] for v in P, i in 1:n] for P in sortOrbit(orb)] for  orb in orbits]
    return Dict{Matrix{Int64}, Vector{Matrix{Int64}}}(orb[1] => orb for orb ∈ newOrbits)
end

function permutahedron(n)
    return orbit_polytope(1:n, symmetric_group(n))
end

function facetsSubPermutahedron(Pi, n)
    #Pi = permutahedron(n) 
    vertsPi = Matrix{Int64}(Pi.pm_polytope.VERTICES)[:,2:n+1]
    nvertsPi = size(vertsPi)[1]
    facetsIndicesBool = Matrix{Int64}(Pi.pm_polytope.VERTICES_IN_FACETS)
    
    fIBrows, fINcols = size(facetsIndicesBool); 
    return  [Set([vertsPi[i,:] for i in 1:nvertsPi if facetsIndicesBool[j,i]==1 ]) for j in 1:fIBrows]
end

function isInternalFacet(F,n)
    externalFacets = facetsSubPermutahedron(permutahedron(n), n)
    if any(map((x) -> issubset(F,x), externalFacets ) )
        return false
    else
        return true
    end
end

function internalFacetsSubPermutahedron(Pi, n)
    facetsPi = facetsSubPermutahedron(Pi, n)
    internalFacetsSets = [F for F in facetsPi if isInternalFacet(F,n)]
    return [[v[i] for v in F, i in 1:n ] for F in internalFacetsSets]
end


function permutation2Flag(p)
    n=length(p)
    irp = reverse(invertPerm(p))
    return [ sort([irp[i] for i in 1:d]) for d in 1:(n-1)]
end


function vertices2Bases(vs, n)
    Bs = Set([])
    nrows,ncols = size(vs)
    
    for i in 1:nrows
        union!(Bs, permutation2Flag(vs[i,:]))
    end
    return Bs    
end

function invertPerm(p)
    n=length(p)
    pd = Dict{Int64,Int64}(i=>p[i] for i in 1:n)
    pdinv = Dict{Int64,Int64}(value=>key for (key,value) in pd)
    [pdinv[i] for i in 1:n]
end


invertPerm (generic function with 1 method)

```n4 = Dict{Int64, Vector{Any}}``` records all matroids on $[4]$, up to symmetry. 

```alln4 = Dict{Int64, Any}``` records all matroids on $[4]$.

```fl4 = Vector{Vector{Set{Set{Int64}}}}``` records all flag matroids on $[4]$.

```allPolytopes4 = Vector{Set{Vector{Int64}}}``` records all flag matroids, but recorded as the vertices of their polytopes.

```repsDict4 = Dict{Set{Vector{Int64}}, Set{Set{Vector{Int64}}}}``` records the flag matroids on $[4]$ up to $\mathfrak{S}_2 \times \mathfrak{S}_4$ symmetry. The keys are the representatives, and the values are the flags in the orbit. 

```Dim3 = Vector{Matrix{Int64}}``` records the flag matroids on $[4]$ whose polytope has dimension 3, up to $\mathfrak{S}_2 \times \mathfrak{S}_4$ symmetry. 

In [5]:
n4 = matroidsBasesFromFile("matroidDatabase/n4.txt",4);
alln4 = allMatroids(n4, 4);
fl4 = flagMatroids(alln4,4);
allPolytopes4 = map((x) -> flagBases2Permutations(x,4), fl4);
repsDict4 = flagMatroidReps(allPolytopes4,4);

In [16]:
sortedRepsDict4 = sortedReps(repsDict4, 4);
sortedReps4 = keys(sortedRepsDict4);
Dims3 = [P for P in sortedReps4 if dim(convex_hull(P)) == 3];

### (polytopes of) rank $(1,2,3)$-flag matroids on $[4]$ and their internal facets 

In the following cells, we record each flag matroid whose polytope is 3-dimensional, as a list of the bases of its constitutients. We also record the flag matroids of each facet not contained in the boundary of the 3-dimensional permutahedron. 

In [8]:
M1 = sortedRepsDict4[Dims3[5]][2]
FacetsM1 = internalFacetsSubPermutahedron(convex_hull(M1), 4)
print("Bases M1 = " * string(sortedFlagMatroidBases(vertices2Bases(M1, 4))) * "\n")
for i in 1:length(FacetsM1)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM1[i], 4)),"\n"))
end

Bases M1 = [[1], [3], [1, 2], [1, 4], [2, 3], [3, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [3], [1, 2], [1, 4], [2, 3], [3, 4], [1, 2, 3], [1, 3, 4]]
Bases Facet 2 = [[1], [3], [1, 2], [2, 3], [1, 2, 3], [1, 2, 4], [2, 3, 4]]
Bases Facet 3 = [[1], [3], [1, 4], [3, 4], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
Bases Facet 4 = [[1], [3], [1, 2], [1, 4], [2, 3], [3, 4], [1, 2, 4], [2, 3, 4]]


In [8]:
M2 = sortedRepsDict4[Dims3[9]][5]
FacetsM2 = internalFacetsSubPermutahedron(convex_hull(M2), 4)
print("Bases M2 = " * string(sortedFlagMatroidBases(vertices2Bases(M2, 4))) * "\n")
for i in 1:length(FacetsM2)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM2[i], 4)),"\n"))
end

Bases M2 = [[1], [2], [1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [2], [1, 2], [1, 4], [2, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]
Bases Facet 2 = [[1], [2], [1, 3], [1, 4], [2, 3], [2, 4], [1, 3, 4], [2, 3, 4]]
Bases Facet 3 = [[1], [2], [1, 3], [2, 3], [1, 2, 3], [1, 3, 4], [2, 3, 4]]


In [9]:
M3 = sortedRepsDict4[Dims3[4]][4]
FacetsM3 = internalFacetsSubPermutahedron(convex_hull(M3), 4)
print("Bases M3 = " * string(sortedFlagMatroidBases(vertices2Bases(M3, 4))) * "\n")
for i in 1:length(FacetsM3)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM3[i], 4)),"\n"))    
end

Bases M3 = [[1], [2], [1, 2], [1, 4], [2, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [2], [1, 2], [1, 4], [2, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]
Bases Facet 2 = [[1], [2], [1, 4], [2, 4], [1, 2, 4], [1, 3, 4], [2, 3, 4]]


In [10]:
M4 = sortedRepsDict4[Dims3[6]][4]
FacetsM4 = internalFacetsSubPermutahedron(convex_hull(M4), 4)
print("Bases M4 = " * string(sortedFlagMatroidBases(vertices2Bases(M4, 4))) * "\n")
for i in 1:length(FacetsM4)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM4[i], 4)),"\n"))
end

Bases M4 = [[1], [2], [4], [1, 2], [2, 4], [1, 2, 3], [1, 2, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [2], [4], [1, 2], [2, 4], [1, 2, 3], [2, 3, 4]]
Bases Facet 2 = [[1], [4], [1, 2], [2, 4], [1, 2, 3], [1, 2, 4], [2, 3, 4]]


In [10]:
M5 = sortedRepsDict4[Dims3[7]][2]
FacetsM5 = internalFacetsSubPermutahedron(convex_hull(M5), 4)
print("Bases M5 = " * string(sortedFlagMatroidBases(vertices2Bases(M5, 4))) * "\n")
for i in 1:length(FacetsM5)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM5[i], 4)),"\n"))
end

Bases M5 = [[1], [3], [1, 2], [1, 3], [1, 4], [2, 3], [3, 4], [1, 2, 3], [1, 3, 4]]
Bases Facet 1 = [[1], [3], [1, 2], [1, 4], [2, 3], [3, 4], [1, 2, 3], [1, 3, 4]]
Bases Facet 1 = [[1], [3], [1, 2], [1, 4], [2, 3], [3, 4], [1, 2, 3], [1, 3, 4]]


In [12]:
M6 = sortedRepsDict4[Dims3[10]][10]
FacetsM6 = internalFacetsSubPermutahedron(convex_hull(M6), 4)
print("Bases M6 = " * string(sortedFlagMatroidBases(vertices2Bases(M6, 4))) * "\n")
for i in 1:length(FacetsM6)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM6[i], 4)),"\n"))
end

Bases M6 = [[1], [3], [4], [1, 2], [1, 4], [2, 3], [2, 4], [3, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [3], [4], [1, 2], [2, 3], [2, 4], [1, 2, 3], [2, 3, 4]]
Bases Facet 2 = [[1], [4], [1, 2], [1, 4], [2, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]
Bases Facet 3 = [[1], [3], [1, 2], [1, 4], [2, 3], [3, 4], [1, 2, 3], [1, 3, 4]]


In [13]:
M7 = sortedRepsDict4[Dims3[2]][3]
FacetsM7 = internalFacetsSubPermutahedron(convex_hull(M7), 4)
print("Bases M7 = " *string(sortedFlagMatroidBases(vertices2Bases(M7, 4))) * "\n")
for i in 1:length(FacetsM7)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM7[i], 4)),"\n"))
end

Bases M7 = [[1], [4], [1, 2], [1, 3], [1, 4], [2, 4], [3, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [4], [1, 2], [1, 3], [2, 4], [3, 4], [1, 2, 3], [2, 3, 4]]
Bases Facet 2 = [[1], [4], [1, 2], [2, 4], [1, 2, 3], [1, 2, 4], [2, 3, 4]]
Bases Facet 3 = [[1], [4], [1, 3], [3, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]


In [14]:
M8 = sortedRepsDict4[Dims3[8]][3]
FacetsM8 = internalFacetsSubPermutahedron(convex_hull(M8), 4)
print("Bases M8 = " * string(sortedFlagMatroidBases(vertices2Bases(M8, 4))) * "\n")
for i in 1:length(FacetsM8)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM8[i], 4)),"\n"))
end

Bases M8 = [[1], [2], [4], [1, 2], [1, 4], [2, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [2], [4], [1, 2], [1, 4], [2, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]


In [15]:
M9 = sortedRepsDict4[Dims3[1]][2]
FacetsM9 = internalFacetsSubPermutahedron(convex_hull(M9), 4)
print("Bases M9 = "); print(sortedFlagMatroidBases(vertices2Bases(M9, 4))); print("\n")
for i in 1:length(FacetsM9)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM9[i], 4)),"\n"))
end

Bases M9 = [[1], [2], [4], [1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [2], [4], [1, 2], [1, 4], [2, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]
Bases Facet 2 = [[1], [2], [4], [1, 3], [2, 3], [3, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]


In [16]:
M10 = sortedRepsDict4[Dims3[15]][7]
FacetsM10 = internalFacetsSubPermutahedron(convex_hull(M10), 4)
print("Bases M10 = " * string(sortedFlagMatroidBases(vertices2Bases(M10, 4))) * "\n")
for i in 1:length(FacetsM10)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM10[i], 4)),"\n"))
end

Bases M10 = [[1], [3], [4], [1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [3], [4], [1, 2], [2, 3], [2, 4], [1, 2, 3], [2, 3, 4]]
Bases Facet 2 = [[1], [4], [1, 2], [1, 4], [2, 4], [1, 2, 3], [1, 3, 4], [2, 3, 4]]


In [17]:
M11 = sortedRepsDict4[Dims3[12]][11]
FacetsM11 = internalFacetsSubPermutahedron(convex_hull(M11), 4)
print("Bases M11 = " * string(sortedFlagMatroidBases(vertices2Bases(M11, 4))) * "\n")
for i in 1:length(FacetsM11)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM11[i], 4)),"\n"))
end

Bases M11 = [[1], [3], [4], [1, 2], [1, 4], [2, 3], [2, 4], [3, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [3], [4], [1, 2], [2, 3], [2, 4], [1, 2, 3], [1, 2, 4], [2, 3, 4]]
Bases Facet 2 = [[1], [3], [1, 2], [1, 4], [2, 3], [3, 4], [1, 2, 3], [1, 3, 4]]


In [24]:
M12 = sortedRepsDict4[Dims3[13]][2]
FacetsM12 = internalFacetsSubPermutahedron(convex_hull(M12), 4)
print("Bases M12 = " * string(sortedFlagMatroidBases(vertices2Bases(M12, 4))) * "\n")
for i in 1:length(FacetsM12)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM12[i], 4)),"\n"))
end

Bases M12 = [[1], [2], [3], [4], [1, 2], [1, 4], [2, 3], [3, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [3], [1, 2], [1, 4], [2, 3], [3, 4], [1, 2, 3], [1, 3, 4]]
Bases Facet 2 = [[2], [4], [1, 2], [1, 4], [2, 3], [3, 4], [1, 2, 4], [2, 3, 4]]


In [19]:
M13 = sortedRepsDict4[Dims3[11]][4]
FacetsM13 = internalFacetsSubPermutahedron(convex_hull(M13), 4)
print("Bases M13 = " * string(sortedFlagMatroidBases(vertices2Bases(M13, 4))) * "\n")
for i in 1:length(FacetsM13)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM13[i], 4)),"\n"))
end

Bases M13 = [[1], [3], [4], [1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [3], [4], [1, 2], [2, 3], [2, 4], [1, 2, 3], [1, 2, 4], [2, 3, 4]]


In [20]:
M14 = sortedRepsDict4[Dims3[14]][5]
FacetsM14 = internalFacetsSubPermutahedron(convex_hull(M14), 4)
print("Bases M14 = " * string(sortedFlagMatroidBases(vertices2Bases(M14, 4))) * "\n")
for i in 1:length(FacetsM14)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM14[i], 4)),"\n"))
end

Bases M14 = [[1], [2], [3], [4], [1, 2], [1, 4], [2, 3], [2, 4], [3, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
Bases Facet 1 = [[1], [3], [1, 2], [1, 4], [2, 3], [3, 4], [1, 2, 3], [1, 3, 4]]


In [21]:
M15 = sortedRepsDict4[Dims3[3]][1]
FacetsM15 = internalFacetsSubPermutahedron(convex_hull(M15), 4)
print("Bases M15 = " * string(sortedFlagMatroidBases(vertices2Bases(M15, 4))) * "\n")
for i in 1:length(FacetsM15)
    print(string("Bases Facet ", i, " = " , sortedFlagMatroidBases(vertices2Bases(FacetsM15[i], 4)),"\n"))
end

Bases M15 = [[1], [2], [3], [4], [1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]
